# Compare Cross Correlations

# of same dataset but different settings

In [1]:
import os
import sys
import time
from datetime import datetime
import glob
import time

import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
import pandas as pd
import pickle

import meshio
from obspy import read
from geotree import gtree
from geotree.utils import convert

In [2]:
#  ------------------ read kernel_filter.txt ------------------
def read_kernel_filter(path, event_name):

    dt = np.dtype([('band', 'S10'), ('type', 'S10'), ('dominant_period', float), ('sigma', float)])
    # import ipdb; ipdb.set_trace()
    kernel_filter_dir = os.path.join(path, event_name, 'outfiles', 'kernel_filter.txt')
    band_info = np.loadtxt(kernel_filter_dir, dtype=dt, skiprows=1, usecols=(0, 1, 2, 3))

    return band_info

#  ------------------ read event pkl ------------------
def read_event_pkl(path):

    ev_load = open(os.path.join(path, 'EVENTS-INFO', 'event_list_pickle'), 'rb')
    ev_pkl = pickle.load(ev_load)
    ev_load.close()

    return ev_pkl

In [3]:
orig_proc_path = '/mnt/seismodata2/MT/_INPUT_/neic_2018_2020_min_mag_7'

path1 = '/mnt/seismodata/MT/SEA-SEIS_TOMO/ffproc/SEA-SEIS_skewed_26.07.2021'
path2 = '/mnt/seismodata/MT/SEA-SEIS_TOMO/ffproc/SEA-SEIS_no_skew_26.07.2021'


path_events = glob.glob(os.path.join(path1, '*.?'))
all_events = []
for i in path_events:
    join = i.split('/')[-1]
    all_events.append(join)
# import ipdb; ipdb.set_trace()
inp_band = read_kernel_filter(path1, all_events[0])
bands_sec = inp_band['dominant_period']

stations = ['DUDE', 'NAMOR', 'LUIGI', 'CHARL','LIGEA', 'LOCH', 'WILSN', 'QUAKY','MAUDE', 'GRAIN', 'GILL', 'TOM', 'EVE', 'BRIAN']


In [4]:
ampstt_dir = os.path.join(path1, event, 'outfiles', 'ffproc.ampstt.band%02d' % 1)

NameError: name 'event' is not defined

In [5]:
header = ['idx','grp','stla','stlo','stazie','stnam','xc_coeff','Tobs','dT','sigma_dT','not_used1','not_used2','A','not_used3','not_used4','not_used5','tB_smgr','tB_mfi','winlen','clip_taumax','not_used6','SNR','ts_step1','ts_step2','cc_step1','cc_step2','clip_step1','clip_step2']
all_df1 = []
all_df2 = []
all_df = []
for i, value in enumerate(bands_sec):
    band = i + 1
    for j, event in enumerate(all_events):
        
        info_pkl = read_event_pkl(orig_proc_path)
        
        try:
            indx = [i for i, ev in enumerate(info_pkl) if ev['event_id'] == event][0]
        except Exception as exp:
            continue
        
        ampstt_dir1 = os.path.join(path1, event, 'outfiles', 'ffproc.ampstt.band%02d' % band)
        ampstt_dir2 = os.path.join(path2, event, 'outfiles', 'ffproc.ampstt.band%02d' % band)
        
        try:
            df1 = pd.read_csv(ampstt_dir1, delim_whitespace=True, skiprows=4, names = header)
            df1.insert(1, "band", [band]*len(df1), True)
            df1.insert(2, "event", [event]*len(df1), True)
            df1.insert(3, "skew", ['skewed']*len(df1), True)
            
            df2 = pd.read_csv(ampstt_dir2, delim_whitespace=True, skiprows=4, names = header)
            df2.insert(1, "band", [band]*len(df2), True)
            df2.insert(2, "event", [event]*len(df2), True)
            df2.insert(3, "skew", ['no_skew']*len(df2), True)
        
        except Exception as exp:
            print(exp, len(df), band)
        
        all_df1.append(df1)
        all_df2.append(df2)
        
        all_df.append(df1)
        all_df.append(df2)
        
        if i == 0:
            break
all_df = pd.concat(all_df)
all_df1 = pd.concat(all_df1)
all_df2 = pd.concat(all_df2)

In [6]:
all_df.sort_values('stnam')
all_df1.sort_values('stnam')
all_df2.sort_values('stnam')

,idx,band,event,skew,grp,stla,stlo,stazie,stnam,xc_coeff,...,winlen,clip_taumax,not_used6,SNR,ts_step1,ts_step2,cc_step1,cc_step2,clip_step1,clip_step2
12,13,8,20200325_024921.a,no_skew,1,49.85,-12.03,81.17,AI.BRIAN.00.HHZ,0.60,...,8.7,0,0.0,1.536,70.0,21.0,0.931,0.597,1,0
12,13,8,20190706_031953.a,no_skew,1,49.85,-12.03,72.41,AI.BRIAN.00.HHZ,0.40,...,8.7,0,0.0,1.642,-66.0,-12.0,0.590,0.397,0,0
13,14,4,20181220_170155.a,no_skew,1,49.85,-12.03,75.38,AI.BRIAN.00.HHZ,0.38,...,19.9,1,0.0,7.329,26.0,30.0,0.941,0.379,0,1
12,13,2,20200213_103344.a,no_skew,1,49.85,-12.03,83.49,AI.BRIAN.00.HHZ,0.95,...,33.7,0,0.0,5.838,70.0,0.0,0.712,0.952,1,0
12,13,4,20200213_103344.a,no_skew,1,49.85,-12.03,83.49,AI.BRIAN.00.HHZ,0.84,...,19.9,0,0.0,1.404,70.0,25.0,0.712,0.841,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,8,20200128_191024.a,no_skew,1,56.34,-24.49,54.45,AI.WILSN.00.HHZ,0.70,...,8.7,0,0.0,0.797,5.0,-19.0,0.869,0.697,0,0
0,1,7,20190222_101723.a,no_skew,1,56.34,-24.49,72.13,AI.WILSN.00.HHZ,0.80,...,10.3,0,0.0,1.655,-15.0,-13.0,0.974,0.795,0,0
5,6,8,20181220_170155.a,no_skew,1,56.34,-24.49,68.67,AI.WILSN.00.HHZ,0.49,...,8.7,0,0.0,1.317,17.0,13.0,0.918,0.495,0,0
1,2,3,20190706_031953.a,no_skew,1,56.34,-24.49,62.69,AI.WILSN.00.HHZ,0.03,...,26.1,1,0.0,5.602,-34.0,30.0,0.623,0.033,0,1


In [8]:
all_df['event'].unique()

array(['20190706_031953.a', '20190526_074115.a', '20181220_170155.a',
       '20200213_103344.a', '20190222_101723.a', '20190301_085042.a',
       '20200128_191024.a', '20181130_172929.a', '20200325_024921.a'], dtype=object)

In [ ]:
# diff = all_df1['dT'] - all_df2['dT']
# df_diff = all_df1[['event', 'band']].copy()
# df_diff.insert(2, 'diff', diff, True)

In [ ]:
# all_df1[['stnam','band', 'dT', 'xc_coeff']].plot.scatter(x='stnam', y='dT', s=100, c='band', cmap='jet');
# all_df2[['event','band', 'dT', 'xc_coeff']].plot.scatter(x='event', y='dT', s=100, c='band', cmap='jet');

In [ ]:
# df1 = all_df1[(all_df1['xc_coeff'] > 0.75) & (all_df1['skew'] == 'skewed')]
# df2 = all_df2[(all_df2['xc_coeff'] > 0.75) & (all_df2['skew'] == 'no_skew')]

In [ ]:
# df1[['band', 'dT', 'xc_coeff']].plot.scatter(x='band', y='dT', s=100, alpha=0.5);
# df2[['band', 'dT', 'xc_coeff']].plot.scatter(x='band', y='dT', s=100, alpha=0.5);

In [ ]:
my_xticks = stations

for i, value in enumerate(bands_sec):
    band = i + 1
    df_combine = []
    for j, event in enumerate(all_events):
        df1_select = all_df1[(all_df1['xc_coeff'] > 0.7)&(all_df1['event']==event)&(all_df1['band']==band)]
        df2_select = all_df2[(all_df1['xc_coeff'] > 0.7)&(all_df2['event']==event)&(all_df2['band']==band)]
        
        df1_select.sort_values('stnam')
        df2_select.sort_values('stnam')
        

        plt.figure()

        ax = df1_select[['stnam', 'dT']].plot.scatter(x='stnam', y='dT', s=100, c='blue',  figsize=(10,5), title=f'Band {band} | {event}')
        df2_select[['stnam','dT']].plot.scatter(x='stnam', y='dT', c='orange', grid=True, s=100, ax=ax)
        plt.xticks(rotation=90)
        
        plt.show()
        
        


In [ ]:
df2_name